In [20]:
import netCDF4
from netCDF4 import Dataset
import gdal

import os 

#from mpl_toolkits.basemap import Basemap

import glob

import pandas as pd

import numpy as np
import rioxarray
import xarray

In [24]:
xds = xarray.open_dataset('GLDASp4_soiltexture_025d.nc4')
#dataset['GLDAS_soiltex'].rio.write_crs('EPSG:4326', inplace = True)
# dataset['GLDAS_soiltex'].rio.to_raster("planet_scope.tif")

xds.rio.set_crs("epsg:4326")

xds['GLDAS_soiltex'].rio.to_raster('xarraytest.tif')

In [37]:
data_nc4 = Dataset('GLDASp4_soiltexture_025d.nc4')

#print(data_nc4)

lats = data_nc4.variables['lat'][:]
lons = data_nc4.variables['lon'][:]
data = data_nc4.variables['GLDAS_soiltex'][0,:,:]

print(len(lats))
print(len(lons))
print(np.shape(data))

# load driver 
driver_tiff = gdal.GetDriverByName('GTiff')

# Create new raster and set the crs
outRaster = 'GLDAS_SoilTypes.tif'
ds = driver_tiff.Create(outRaster, xsize = len(lons), \
                    ysize = len(lats), bands =1,\
                    eType = gdal.GDT_Float32)

# top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
#ds.SetGeoTransform([lons[0],0.25,0,lats[0],0,-0.25])
ds.SetGeoTransform([-180,0.25,0,90,0,-0.25])
ds.SetProjection('EPSG:4326')

# Write the values to saved raster
ds.GetRasterBand(1).WriteArray(np.flipud(data))

ds = None

print('saved '+outRaster)

600
1440
(600, 1440)
saved GLDAS_SoilTypes.tif


In [28]:
print(lons[0])
print(lats[0])

-179.875
-59.875


In [43]:
#ldas.gsfc.nasa.gov/gldas/soils#
#https://wiki.tuflow.com/index.php?title=Green-Ampt_Infiltration_Parameters

#### Create rasters for required soil parameters
dims = (np.shape(data))

suction = np.empty(np.shape(data))
hydraulicConductivity = np.empty(np.shape(data))
porosity = np.empty(np.shape(data))
print(dims[0])

for i in range((dims[0])):
    for j in range(dims[1]):
        #print(data[i,j])
        # sand
        if (data[i,j] == 1):
            suction[i,j] = 49.5
            hydraulicConductivity[i,j] = 117.8
            porosity[i,j] = 0.417
            
        # Loamy Sand
        elif (data[i,j] == 2):
            suction[i,j] = 61.3
            hydraulicConductivity[i,j] = 29.9
            porosity[i,j] = 0.401
            
        # Sandy Loam
        elif (data[i,j] == 3):
            suction[i,j] = 110.1
            hydraulicConductivity[i,j] = 10.9
            porosity[i,j] = 0.412
        
        # Silt Loam
        elif (data[i,j] == 4):
            suction[i,j] = 166.8
            hydraulicConductivity[i,j] = 3.4
            porosity[i,j] = 0.486
            
        # Silt ##
        elif (data[i,j] == 5):
            suction[i,j] = 200
            hydraulicConductivity[i,j] = 3
            porosity[i,j] = 0.42
            
        # Loam
        elif (data[i,j] == 6):
            suction[i,j] = 88.9
            hydraulicConductivity[i,j] = 7.6
            porosity[i,j] = 0.434
            
        # Sandy Clay Loam
        elif (data[i,j] == 7):
            suction[i,j] = 218.5
            hydraulicConductivity[i,j] = 1.5
            porosity[i,j] = 0.33
            
        # Silty Clay Loam
        elif (data[i,j] == 8):
            suction[i,j] = 273
            hydraulicConductivity[i,j] = 1
            porosity[i,j] = 0.432
            
        # Clay Loam
        elif (data[i,j] == 9):
            suction[i,j] = 208.8
            hydraulicConductivity[i,j] = 1
            porosity[i,j] = 0.309
            
        # Sandy Clay
        elif (data[i,j] == 10):
            suction[i,j] = 239
            hydraulicConductivity[i,j] = 0.6
            porosity[i,j] = 0.321
            
        # Silty Clay 
        elif (data[i,j] == 11):
            suction[i,j] = 292.2
            hydraulicConductivity[i,j] = 0.5
            porosity[i,j] = 0.423
            
        # Clay
        elif (data[i,j] == 12):
            suction[i,j] = 316.3
            hydraulicConductivity[i,j] = 0.3
            porosity[i,j] = 0.385
            
        # Organic Materials ##
        elif (data[i,j] == 13):
            suction[i,j] = 320
            hydraulicConductivity[i,j] = 0.5
            porosity[i,j] = 0.4
        
        else:
            suction[i,j] = 0
            hydraulicConductivity[i,j] = 0
            porosity[i,j] = 0   
            
###### Saving the properties into rasters ######
## Create new raster and set the crs
suction = suction/1000
outRaster = 'suction_m.tif'
ds = driver_tiff.Create(outRaster, xsize = len(lons), \
                    ysize = len(lats), bands =1,\
                    eType = gdal.GDT_Float32)

# top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
ds.SetGeoTransform([-180,0.25,0,90,0,-0.25])
ds.SetProjection('EPSG:4326')

# Write the values to saved raster
ds.GetRasterBand(1).WriteArray(np.flipud(suction))

ds = None

print('saved '+outRaster)

## Create new raster and set the crs
hydraulicConductivity = hydraulicConductivity/(1000*3600)
outRaster = 'hydraulicConductivity_ms-1.tif'
ds = driver_tiff.Create(outRaster, xsize = len(lons), \
                    ysize = len(lats), bands =1,\
                    eType = gdal.GDT_Float32)

# top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
ds.SetGeoTransform([-180,0.25,0,90,0,-0.25])
ds.SetProjection('EPSG:4326')

# Write the values to saved raster
ds.GetRasterBand(1).WriteArray(np.flipud(hydraulicConductivity))

ds = None

print('saved '+outRaster)


## Create new raster and set the crs
outRaster = 'porosity.tif'
ds = driver_tiff.Create(outRaster, xsize = len(lons), \
                    ysize = len(lats), bands =1,\
                    eType = gdal.GDT_Float32)

# top left x, w-e pixel resolution, rotation, top left y, rotation, n-s pixel resolution
ds.SetGeoTransform([-180,0.25,0,90,0,-0.25])
ds.SetProjection('EPSG:4326')

# Write the values to saved raster
ds.GetRasterBand(1).WriteArray(np.flipud(porosity))

ds = None

print('saved '+outRaster)

600
saved suction_m.tif
saved hydraulicConductivity_ms-1.tif
saved porosity.tif
